In [ ]:
!pip install loguru

/Users/mileshe/Desktop/NLP_Project/COMP90042_2025


In [ ]:
import os
import sys
print(os.getcwd())
sys.path.append('Sim_tools')

from Sim_tools.dataset_Sim import TrainDataset, TestDataset
from Sim_tools.model_Sim import SimcseModel, simcse_sup_loss, simcse_unsup_loss
from Sim_tools.train_Sim import load_train_data_supervised, train_sup
from Sim_tools.embed_evidence import embed_evidence

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import BertModel, BertConfig, BertTokenizer
from loguru import logger
from tqdm import tqdm

import pandas as pd
import json

In [ ]:
batch_size = 16
max_length = 256
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

print(f"Using device: {device}")
checkpoint = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(checkpoint)
model = SimcseModel(pretrained_model=checkpoint, pooling='pooler', dropout=0.1).to(device)
model.load_state_dict(torch.load("saved_model/best_model.pt", map_location=device))
model.eval()

    
evidence_csv_path = "data/evidence.json"  
output_csv_path = "data/evidence_embed.json" 
embed_evidence(evidence_csv_path, output_csv_path, model, tokenizer, device, max_length)